<a href="https://colab.research.google.com/github/ethangourley17-dev/12345/blob/main/Copy_of_Roof_measurement_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roof Measurement AI - Sales & Estimation Tool

## Project Overview

This notebook provides an AI-powered roofing sales and estimation platform that:
- **Calculates roof areas** with pitch correction
- **Assesses damage severity** and adjusts costs accordingly
- **Estimates project costs** including removal, materials, and labor
- **Compares material options** (Asphalt, Metal, Clay Tile)
- **Analyzes ROI** including property value increases and solar potential
- **Generates professional HTML reports** for client presentations

## Key Features

✅ **Multi-Structure Support** - Handle properties with multiple buildings (house, garage, etc.)

✅ **Interactive Widgets** - Visual roof calculator with real-time cost updates

✅ **Professional Reports** - Styled HTML reports with damage assessment and cost breakdowns

✅ **Material Comparison** - Compare costs, lifespan, warranty, and wind ratings

✅ **Solar ROI Analysis** - Calculate long-term savings from solar panel installation

## How to Use This Notebook

1. **Run Dependencies Section** - Install required packages
2. **Create Module Files** - Define the core calculation engines
3. **Try Interactive Demo** - Use widgets to explore different roof configurations
4. **Generate Sample Report** - Create a complete HTML report for a sample property
5. **Customize for Your Needs** - Modify parameters and generate your own reports

## Technical Details

**Cost Calculation Formula:**
```
actual_area = footprint × pitch_factor × (1 + waste%)
squares = actual_area / 100
total_cost = (removal + installation) × squares
```

**Damage Multipliers:**
- Intact: 1.0x (< 30% probability)
- Minor Damage: 1.2x (30-70% probability)
- Severe Damage: 1.5x (> 70% probability)

**Material Cost Multipliers:**
- Asphalt Shingle: 1.0x baseline
- Metal Seam: 2.5x
- Clay Tile: 3.0x


---
## 1. Dependencies Installation

Install required Python packages for visualization and widgets.

**Required packages:**
- `matplotlib` - Plotting and visualization
- `numpy` - Numerical calculations
- `ipywidgets` - Interactive controls


In [ ]:
# Install dependencies
!pip install -q matplotlib numpy ipywidgets

print("✅ Dependencies installed successfully")

---
## 2. Module Definitions

Create the core module files that contain our calculation engines.
These modules provide clean, reusable code for:
- **sales_tools.py** - Cost estimation, damage assessment, ROI analysis
- **measurement_engine.py** - Roof area calculations and structure identification
- **report_builder.py** - HTML report generation


### 2.1 Create sales_tools.py

This module contains classes for:
- **DamageAssessor** - Evaluate roof condition and calculate damage multipliers
- **CostEstimator** - Calculate removal, material, and labor costs
- **MaterialComparator** - Compare different roofing materials
- **ROIAnalyzer** - Calculate property value increases and solar savings


In [ ]:
%%writefile sales_tools.py
"""
Sales Tools Module for Roof Measurement AI

This module provides classes for damage assessment, cost estimation, 
material comparison, and ROI analysis for roofing projects.
"""

import math


class DamageAssessor:
    """Assess roof damage severity and calculate damage multipliers."""
    
    def assess(self, damage_probability):
        """
        Assess damage category and severity factor.
        
        Args:
            damage_probability (float): Probability of damage (0.0 to 1.0)
        
        Returns:
            dict: Category and severity_factor
        """
        if damage_probability < 0.3:
            return {'category': 'Intact', 'severity_factor': 1.0}
        elif damage_probability < 0.7:
            return {'category': 'Minor Damage', 'severity_factor': 1.2}
        else:
            return {'category': 'Severe Damage', 'severity_factor': 1.5}


class CostEstimator:
    """Calculate detailed cost estimates for roof replacement projects."""
    
    def __init__(self, removal_price_per_sq=50.0, install_price_per_sq=350.0):
        """
        Initialize with pricing parameters.
        
        Args:
            removal_price_per_sq (float): Cost per square for removal
            install_price_per_sq (float): Cost per square for installation
        """
        self.removal_price_per_sq = removal_price_per_sq
        self.install_price_per_sq = install_price_per_sq
    
    def calculate_estimate(self, area_sqft, pitch_factor=1.0, waste_percent=10.0, removal_multiplier=1.0):
        """
        Calculate roof replacement cost estimate.
        
        Formula:
            actual_area = area × pitch_factor × (1 + waste%/100)
            squares = actual_area / 100
            total_cost = (removal + installation) × squares
        
        Args:
            area_sqft (float): Roof area in square feet
            pitch_factor (float): Multiplier based on roof pitch
            waste_percent (float): Material waste percentage
            removal_multiplier (float): Damage severity multiplier
        
        Returns:
            dict: Detailed cost breakdown
        """
        # Calculate actual area accounting for pitch and waste
        actual_area = area_sqft * pitch_factor * (1 + waste_percent / 100.0)
        squares = actual_area / 100.0
        
        # Calculate costs
        removal_cost = squares * self.removal_price_per_sq * removal_multiplier
        total_install_cost = squares * self.install_price_per_sq
        material_cost = total_install_cost * 0.40  # 40% material
        labor_cost = total_install_cost * 0.60     # 60% labor
        total_cost = removal_cost + total_install_cost
        
        return {
            'removal_cost': round(removal_cost, 2),
            'material_cost': round(material_cost, 2),
            'labor_cost': round(labor_cost, 2),
            'total_cost': round(total_cost, 2),
            'squares': round(squares, 2)
        }


class MaterialComparator:
    """Compare costs and specs across different roofing materials."""
    
    def compare(self, base_cost):
        """
        Compare material options based on base cost.
        
        Args:
            base_cost (float): Base cost (asphalt shingle)
        
        Returns:
            list: Material comparison data
        """
        return [
            {
                'material_name': 'Asphalt Shingle',
                'estimated_cost': round(base_cost * 1.0, 2),
                'lifespan_years': 20,
                'warranty_years': 15,
                'wind_rating_mph': 110
            },
            {
                'material_name': 'Metal Seam',
                'estimated_cost': round(base_cost * 2.5, 2),
                'lifespan_years': 50,
                'warranty_years': 40,
                'wind_rating_mph': 140
            },
            {
                'material_name': 'Clay Tile',
                'estimated_cost': round(base_cost * 3.0, 2),
                'lifespan_years': 75,
                'warranty_years': 50,
                'wind_rating_mph': 150
            }
        ]


class ROIAnalyzer:
    """Calculate return on investment metrics for roofing projects."""
    
    def calculate_property_value_increase(self, total_project_cost):
        """
        Calculate estimated property value increase.
        
        Industry estimates: 60-70% cost recovery
        
        Args:
            total_project_cost (float): Total project cost
        
        Returns:
            dict: Low and high estimates
        """
        return {
            'low_estimate': round(total_project_cost * 0.60, 2),
            'high_estimate': round(total_project_cost * 0.70, 2)
        }
    
    def calculate_solar_roi(self, annual_sunlight_hours, solar_capacity_kw, electricity_rate=0.13):
        """
        Calculate solar panel ROI metrics.
        
        Formula:
            annual_energy = sunlight_hours × capacity × efficiency(0.8)
            annual_savings = annual_energy × electricity_rate
        
        Args:
            annual_sunlight_hours (float): Average annual sunlight
            solar_capacity_kw (float): Solar capacity in kW
            electricity_rate (float): Cost per kWh
        
        Returns:
            dict: Solar ROI calculations
        """
        efficiency = 0.80
        annual_energy_kwh = annual_sunlight_hours * solar_capacity_kw * efficiency
        annual_savings = annual_energy_kwh * electricity_rate
        twenty_year_savings = annual_savings * 20
        
        return {
            'annual_savings': round(annual_savings, 2),
            'twenty_year_savings': round(twenty_year_savings, 2),
            'annual_energy_kwh': round(annual_energy_kwh, 2)
        }


### 2.2 Create measurement_engine.py

This module handles roof measurements and calculations:
- **calculate_roof_area()** - Calculate actual roof area with pitch correction
- **calculate_pitch_factor()** - Convert pitch angle to area multiplier
- **identify_structures()** - Mock function for structure identification


In [ ]:
%%writefile measurement_engine.py
"""
Measurement Engine Module for Roof Measurement AI

Provides functions for roof area calculations and structure identification.
"""

import math


def calculate_roof_area(footprint_sqft, pitch_degrees):
    """
    Calculate actual roof area based on footprint and pitch.
    
    Formula: area = footprint × (1 / cos(pitch))
    
    Args:
        footprint_sqft (float): Horizontal footprint area
        pitch_degrees (float): Roof pitch angle in degrees
    
    Returns:
        float: Actual roof surface area in square feet
    """
    pitch_factor = 1.0 / math.cos(math.radians(pitch_degrees))
    return footprint_sqft * pitch_factor


def calculate_pitch_factor(pitch_degrees):
    """
    Calculate pitch factor from pitch angle.
    
    Args:
        pitch_degrees (float): Roof pitch angle
    
    Returns:
        float: Pitch factor (area multiplier)
    """
    return 1.0 / math.cos(math.radians(pitch_degrees))


def identify_structures(lat, lon):
    """
    Simulate identifying structures at a location.
    
    In production, this would query building databases or
    use computer vision on satellite imagery.
    
    Args:
        lat (float): Latitude
        lon (float): Longitude
    
    Returns:
        list: Structure data with roof area and pitch
    """
    return [
        {
            "name": "Main House",
            "roof_area_sqft": 2800,
            "roof_pitch_degrees": 30
        },
        {
            "name": "Detached Garage",
            "roof_area_sqft": 600,
            "roof_pitch_degrees": 15
        }
    ]


### 2.3 Create report_builder.py

This module generates professional HTML reports with:
- Color-coded damage assessment
- Structure-by-structure cost breakdown
- Material comparison tables
- ROI and solar analysis
- Responsive CSS styling


In [ ]:
%%writefile report_builder.py
"""
Report Builder Module for Roof Measurement AI

Generates professional HTML reports for roofing projects.
"""

from datetime import datetime


class ReportBuilder:
    """Generate professional HTML reports for roofing estimates."""
    
    def add_satellite_overlay(self, image_base64):
        """Generate HTML for satellite image overlay."""
        if image_base64:
            return f'<div class="satellite-view"><img src="data:image/png;base64,{image_base64}" alt="Satellite View"></div>'
        else:
            return '<div class="satellite-view">Satellite imagery unavailable</div>'
    
    def create_pdf(self, html_content, filename):
        """Placeholder for PDF generation."""
        print(f"[Mock] Generating PDF report: {filename}")
        return filename
    
    def generate_html(self, scenario_data, structure_results, grand_total_cost, 
                     grand_total_squares, property_value, solar_roi, 
                     damage_assessment, image_base64=None):
        """
        Generate complete HTML report.
        
        Args:
            scenario_data: Property information
            structure_results: List of structure estimates
            grand_total_cost: Total project cost
            grand_total_squares: Total roofing squares
            property_value: Property value estimates
            solar_roi: Solar ROI calculations
            damage_assessment: Damage category and severity
            image_base64: Optional satellite image
        
        Returns:
            str: Complete HTML document
        """
        img_html = self.add_satellite_overlay(image_base64)
        
        # Determine status color
        category = damage_assessment.get('category', 'Unknown')
        status_class = 'status-unknown'
        if category == 'Intact':
            status_class = 'status-intact'
        elif category == 'Minor Damage':
            status_class = 'status-minor'
        elif category == 'Severe Damage':
            status_class = 'status-severe'
        
        # CSS Styling
        css = """
        <style>
            body { font-family: 'Helvetica Neue', Arial, sans-serif; color: #1e293b; max-width: 1000px; margin: 0 auto; padding: 40px; background-color: white; }
            .header { border-bottom: 4px solid #2563eb; padding-bottom: 20px; margin-bottom: 40px; display: flex; justify-content: space-between; align-items: flex-end; }
            .header h1 { color: #2563eb; margin: 0; font-size: 28px; }
            .header p { color: #64748b; margin: 0; font-size: 14px; }
            .title-section { background: linear-gradient(135deg, #2563eb 0%, #1e40af 100%); color: white; padding: 30px; border-radius: 12px; margin-bottom: 30px; display: flex; align-items: center; gap: 20px; }
            .title-content { flex: 1; }
            .title-section h2 { margin: 0 0 10px 0; font-size: 32px; }
            .satellite-view { width: 300px; height: 200px; border-radius: 8px; border: 3px solid white; background-color: #cbd5e1; display: flex; align-items: center; justify-content: center; color: #1e293b; font-size: 0.8em; text-align: center; overflow: hidden; }
            .satellite-view img { width: 100%; height: 100%; object-fit: cover; }
            .section { margin-bottom: 50px; }
            .section h2 { border-bottom: 2px solid #e2e8f0; padding-bottom: 15px; color: #1e293b; margin-bottom: 25px; font-size: 24px; }
            .assessment-box { padding: 20px; border-radius: 8px; margin-bottom: 40px; border-left: 6px solid #ccc; background-color: #f8fafc; }
            .status-intact { border-left-color: #22c55e; background-color: #f0fdf4; }
            .status-minor { border-left-color: #f97316; background-color: #fff7ed; }
            .status-severe { border-left-color: #ef4444; background-color: #fef2f2; }
            .assessment-title { font-weight: bold; font-size: 1.2em; margin-bottom: 10px; color: #334155; }
            .assessment-detail { font-size: 1em; color: #475569; }
            .structure-block { background: #f8fafc; border: 1px solid #e2e8f0; border-radius: 8px; padding: 20px; margin-bottom: 20px; }
            .structure-title { font-size: 20px; font-weight: bold; color: #1e293b; margin-bottom: 15px; border-left: 4px solid #2563eb; padding-left: 10px; }
            .grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px; }
            .box { background: white; padding: 15px; border-radius: 6px; box-shadow: 0 1px 3px rgba(0,0,0,0.05); }
            .box h3 { color: #64748b; font-size: 0.75em; text-transform: uppercase; margin: 0 0 5px 0; }
            .box .value { font-size: 1.25em; font-weight: 700; color: #1e293b; }
            .grand-total-box { background: #f0fdf4; border: 2px solid #16a34a; padding: 25px; border-radius: 12px; text-align: center; margin-top: 30px; }
            .grand-total-box h3 { color: #166534; margin: 0 0 10px 0; }
            .grand-total-box .value { font-size: 3em; font-weight: 800; color: #15803d; }
            .roi-section { display: flex; gap: 20px; margin-top: 30px; }
            .roi-card { flex: 1; background: #fffbeb; border: 1px solid #fcd34d; padding: 20px; border-radius: 8px; }
            table { width: 100%; border-collapse: collapse; margin-top: 10px; font-size: 0.9em; }
            th, td { text-align: left; padding: 8px; border-bottom: 1px solid #e2e8f0; }
            th { background-color: #f1f5f9; }
        </style>
        """
        
        # Build HTML content
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Report - {scenario_data['address']}</title>
            <meta charset="UTF-8">
            {css}
        </head>
        <body>
            <div class="header">
                <div><h1>Roof Measure AI</h1><p>Multi-Structure Analysis</p></div>
                <div><p>{datetime.now().strftime('%B %d, %Y')}</p></div>
            </div>
            <div class="title-section">
                <div class="title-content">
                    <h2>{scenario_data['address']}</h2>
                    <p>Comprehensive Roofing Assessment</p>
                </div>
                {img_html}
            </div>
            <div class="section">
                <h2>Damage Assessment</h2>
                <div class="assessment-box {status_class}">
                    <div class="assessment-title">Status: {damage_assessment['category']}</div>
                    <div class="assessment-detail">Severity Factor: {damage_assessment['severity_factor']}x</div>
                </div>
            </div>
            <div class="section">
                <h2>Structure Analysis</h2>
        """
        
        # Add structures
        for structure in structure_results:
            est = structure['estimates']
            html_content += f"""
                <div class="structure-block">
                    <div class="structure-title">{structure['name']}</div>
                    <div class="grid">
                        <div class="box"><h3>Total Cost</h3><div class="value">${est['total_cost']:,.2f}</div></div>
                        <div class="box"><h3>Removal</h3><div class="value">${est['removal_cost']:,.2f}</div></div>
                        <div class="box"><h3>Material</h3><div class="value">${est['material_cost']:,.2f}</div></div>
                        <div class="box"><h3>Labor</h3><div class="value">${est['labor_cost']:,.2f}</div></div>
                    </div>
                    <h3>Material Options</h3>
                    <table><thead><tr><th>Material</th><th>Cost</th><th>Lifespan</th><th>Warranty</th><th>Wind Rating</th></tr></thead><tbody>
            """
            for comp in structure['comparisons']:
                html_content += f"<tr><td><strong>{comp['material_name']}</strong></td><td>${comp['estimated_cost']:,.2f}</td><td>{comp['lifespan_years']} years</td><td>{comp['warranty_years']} years</td><td>{comp['wind_rating_mph']} mph</td></tr>"
            html_content += "</tbody></table></div>"
        
        # Grand total and ROI
        html_content += f"""
                <div class="grand-total-box">
                    <h3>Total Project Investment</h3>
                    <div class="value">${grand_total_cost:,.2f}</div>
                    <p style="margin-top: 10px; color: #166534;">Total: {grand_total_squares} squares</p>
                </div>
            </div>
            <div class="section">
                <h2>Return on Investment</h2>
                <div class="roi-section">
                    <div class="roi-card">
                        <h3>Property Value Increase</h3>
                        <p style="font-size: 1.5em; font-weight: bold;">${property_value['low_estimate']:,.2f} - ${property_value['high_estimate']:,.2f}</p>
                    </div>
                    <div class="roi-card">
                        <h3>Solar Potential</h3>
                        <p style="font-size: 1.5em; font-weight: bold;">${solar_roi['annual_savings']:,.2f}/year</p>
                        <p>20-year: ${solar_roi['twenty_year_savings']:,.2f}</p>
                    </div>
                </div>
            </div>
        </body></html>
        """
        
        return html_content


---
## 3. Import Modules

Import the modules we just created and verify they work correctly.


In [ ]:
# Import our custom modules
import math
import sales_tools
import measurement_engine
import report_builder

# Quick verification
print("✅ Module imports successful")

# Test basic functionality
assessor = sales_tools.DamageAssessor()
result = assessor.assess(0.5)
print(f"✅ DamageAssessor test: {result['category']}")

area = measurement_engine.calculate_roof_area(2000, 30)
print(f"✅ Measurement engine test: {area:.2f} sqft")

builder = report_builder.ReportBuilder()
print("✅ ReportBuilder instantiated")

---
## 4. Interactive Roof Calculator

Use this interactive widget to explore different roof configurations.

**Features:**
- Adjust length, width, and pitch with sliders
- See real-time cost calculations
- View roof profile and cost breakdown visualizations

**Instructions:**
1. Move the sliders to adjust roof dimensions and pitch
2. Watch the visualizations update in real-time
3. View the cost summary below the charts


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider
from IPython.display import display

# Initialize estimator
estimator = sales_tools.CostEstimator()

def roof_calculator(length=50, width=40, pitch=25):
    """
    Interactive roof calculator with visualization.
    
    Args:
        length (float): Roof length in feet
        width (float): Roof width in feet
        pitch (float): Roof pitch in degrees
    """
    # Calculate areas
    footprint_area = length * width
    actual_area = measurement_engine.calculate_roof_area(footprint_area, pitch)
    pitch_factor = measurement_engine.calculate_pitch_factor(pitch)
    
    # Calculate costs
    estimate = estimator.calculate_estimate(
        area_sqft=actual_area,
        pitch_factor=1.0,  # Already factored into actual_area
        removal_multiplier=1.0
    )
    
    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left panel: Roof profile side view
    ax1.set_xlim(0, 100)
    ax1.set_ylim(0, 50)
    ax1.set_aspect('equal')
    ax1.set_title('Roof Profile (Side View)', fontsize=14, fontweight='bold')
    
    # Draw roof profile
    base_width = 80
    base_x = 10
    height = base_width / 2 * np.tan(np.radians(pitch))
    
    # Roof lines
    roof_x = [base_x, base_x + base_width/2, base_x + base_width]
    roof_y = [10, 10 + height, 10]
    ax1.plot(roof_x, roof_y, 'b-', linewidth=3, label='Roof Surface')
    ax1.plot([base_x, base_x + base_width], [10, 10], 'k--', linewidth=2, label='Footprint')
    
    # Add dimension labels
    ax1.text(base_x + base_width/2, 5, f'{length} ft', ha='center', fontsize=10)
    ax1.text(base_x + base_width/2, 15 + height, f'{pitch}°', ha='center', fontsize=12, fontweight='bold')
    
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlabel('Distance (ft)')
    ax1.set_ylabel('Height (ft)')
    
    # Right panel: Cost breakdown pie chart
    costs = [estimate['removal_cost'], estimate['material_cost'], estimate['labor_cost']]
    labels = ['Removal', 'Materials', 'Labor']
    colors = ['#ef4444', '#3b82f6', '#22c55e']
    
    ax2.pie(costs, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Cost Breakdown', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    separator = "="*60
    print(separator)
    print("ROOF CALCULATOR RESULTS")
    print(separator)
    print(f"Dimensions:       {length} ft × {width} ft")
    print(f"Footprint Area:   {footprint_area:,.0f} sq ft")
    print(f"Pitch:            {pitch}°")
    print(f"Pitch Factor:     {pitch_factor:.3f}x")
    print(f"Actual Area:      {actual_area:,.0f} sq ft")
    print(f"Roofing Squares:  {estimate['squares']}")
    print("-"*60)
    print(f"Removal Cost:     ${estimate['removal_cost']:>10,.2f}")
    print(f"Material Cost:    ${estimate['material_cost']:>10,.2f}")
    print(f"Labor Cost:       ${estimate['labor_cost']:>10,.2f}")
    print(separator)
    print(f"TOTAL COST:       ${estimate['total_cost']:>10,.2f}")
    print(separator)

# Create interactive widget
interact(
    roof_calculator,
    length=FloatSlider(min=20, max=100, step=5, value=50, description='Length (ft):'),
    width=FloatSlider(min=20, max=100, step=5, value=40, description='Width (ft):'),
    pitch=FloatSlider(min=5, max=45, step=5, value=25, description='Pitch (°):')
);


---
## 5. Complete Example: Generate Professional Report

This section demonstrates a complete end-to-end workflow:

1. **Define property scenario** - Address, location, damage assessment
2. **Identify structures** - Main house and garage
3. **Calculate costs** - For each structure with damage multipliers
4. **Compare materials** - Asphalt, metal, clay tile options
5. **Analyze ROI** - Property value increase and solar potential
6. **Generate HTML report** - Professional styled output

The generated report will be saved as an HTML file that can be:
- Opened in any web browser
- Shared with clients
- Printed or converted to PDF


In [ ]:
print("Starting complete report generation example...")
sep_line = "="*70
print(sep_line)

# ============================================================================
# STEP 1: Define Scenario Data
# ============================================================================

scenario_data = {
    "address": "51046 Range Road 224",
    "lat": 53.4357,
    "lon": -113.2185,
    "annual_sunlight_hours": 2200,
    "solar_panel_capacity_kw": 10.5,
    "damage_probability": 0.85  # High damage probability
}

print(f"Property: {scenario_data['address']}")
print(f"Location: {scenario_data['lat']}, {scenario_data['lon']}")
print(f"Damage Probability: {scenario_data['damage_probability']*100}%")

# ============================================================================
# STEP 2: Assess Damage
# ============================================================================

assessor = sales_tools.DamageAssessor()
damage_assessment = assessor.assess(scenario_data["damage_probability"])

print(f"\nDamage Assessment: {damage_assessment['category']}")
print(f"Severity Factor: {damage_assessment['severity_factor']}x")

# ============================================================================
# STEP 3: Identify Structures
# ============================================================================

structures = measurement_engine.identify_structures(
    scenario_data["lat"], 
    scenario_data["lon"]
)

print(f"\nStructures Identified: {len(structures)}")
for s in structures:
    print(f"  - {s['name']}: {s['roof_area_sqft']} sqft @ {s['roof_pitch_degrees']}°")

# ============================================================================
# STEP 4: Calculate Costs for Each Structure
# ============================================================================

estimator = sales_tools.CostEstimator()
comparator = sales_tools.MaterialComparator()

structure_results = []
grand_total_cost = 0.0
grand_total_squares = 0.0

print("\nCalculating costs for each structure...")

for structure in structures:
    # Calculate pitch factor
    pitch_factor = measurement_engine.calculate_pitch_factor(
        structure['roof_pitch_degrees']
    )
    
    # Estimate costs with damage multiplier
    estimates = estimator.calculate_estimate(
        structure['roof_area_sqft'],
        pitch_factor=pitch_factor,
        removal_multiplier=damage_assessment['severity_factor']
    )
    
    # Compare materials
    comparisons = comparator.compare(estimates['total_cost'])
    
    structure_results.append({
        "name": structure['name'],
        "estimates": estimates,
        "comparisons": comparisons
    })
    
    grand_total_cost += estimates['total_cost']
    grand_total_squares += estimates['squares']
    
    print(f"  {structure['name']}: ${estimates['total_cost']:,.2f} ({estimates['squares']} squares)")

print(f"\nGrand Total: ${grand_total_cost:,.2f}")
print(f"Total Squares: {grand_total_squares}")

# ============================================================================
# STEP 5: Calculate ROI
# ============================================================================

roi_analyzer = sales_tools.ROIAnalyzer()

property_value = roi_analyzer.calculate_property_value_increase(grand_total_cost)
solar_roi = roi_analyzer.calculate_solar_roi(
    scenario_data["annual_sunlight_hours"],
    scenario_data["solar_panel_capacity_kw"]
)

print("\nROI Analysis:")
print(f"  Property Value Increase: ${property_value['low_estimate']:,.2f} - ${property_value['high_estimate']:,.2f}")
print(f"  Solar Annual Savings: ${solar_roi['annual_savings']:,.2f}")
print(f"  Solar 20-Year Savings: ${solar_roi['twenty_year_savings']:,.2f}")

# ============================================================================
# STEP 6: Generate HTML Report
# ============================================================================

builder = report_builder.ReportBuilder()

html_content = builder.generate_html(
    scenario_data,
    structure_results,
    grand_total_cost,
    grand_total_squares,
    property_value,
    solar_roi,
    damage_assessment,
    image_base64=None  # No satellite image in this example
)

# Save report to file
output_filename = "Roof_Estimate_Report.html"
with open(output_filename, "w") as f:
    f.write(html_content)

print(f"\n{sep_line}")
print(f"✅ SUCCESS! Report saved as: {output_filename}")
print(sep_line)
print("\nNext steps:")
print("1. Download the HTML file from the file browser (left sidebar)")
print("2. Open it in your web browser to view the professional report")
print("3. Share with clients or convert to PDF if needed")


---
## 6. Usage Notes & Customization

### How to Customize the Report

**Modify pricing:**
```python
estimator = CostEstimator(
    removal_price_per_sq=60.0,  # Increase removal price
    install_price_per_sq=400.0   # Increase install price
)
```

**Change material multipliers:**
Edit the `MaterialComparator.compare()` method to adjust material cost ratios.

**Add more structures:**
Edit the `identify_structures()` function to return additional buildings.

**Adjust damage thresholds:**
Modify the `DamageAssessor.assess()` method to change probability ranges.

### Formulas Reference

**Pitch Factor Calculation:**
```
pitch_factor = 1 / cos(pitch_in_radians)
```

**Actual Roof Area:**
```
actual_area = footprint_area × pitch_factor × (1 + waste%/100)
```

**Total Cost:**
```
squares = actual_area / 100
removal = squares × removal_price × damage_multiplier
install = squares × install_price
total = removal + install
```

### Tips for Production Use

1. **Add error handling** - Wrap calculations in try/except blocks
2. **Validate inputs** - Check for negative values and reasonable ranges
3. **Add logging** - Track calculations for audit purposes
4. **Database integration** - Store results for historical tracking
5. **API integration** - Connect to real satellite imagery services
6. **PDF export** - Use weasyprint or pdfkit for PDF generation

### Next Steps

- Integrate with Google Maps Static API for real satellite imagery
- Add computer vision for automated pitch detection
- Connect to building footprint databases
- Implement PDF export functionality
- Add user authentication and report storage
- Create mobile-responsive web interface


---
## Summary

✅ **Modules Created:** sales_tools.py, measurement_engine.py, report_builder.py

✅ **Interactive Demo:** Working roof calculator with visualizations

✅ **Complete Example:** Full report generation workflow

✅ **Professional Output:** HTML report with styling and calculations

### What This Notebook Provides

This reorganized notebook gives you:
- **Clean modular architecture** with separate, well-documented files
- **Interactive exploration** of roof configurations and costs
- **Production-ready components** for cost estimation and reporting
- **Extensible framework** for adding new features

### Key Capabilities

- Multi-structure property support
- Damage assessment with cost adjustments
- Material comparison (Asphalt, Metal, Clay)
- ROI analysis (property value + solar)
- Professional HTML report generation

### Credits

**Roof Measurement AI** - Sales & Estimation Tool

Reorganized and documented for clarity and maintainability.
